# Eurostat bioenergy balance 2018

Extract bioenergy related data from an archive containing XLSB files, one for each EU country which contain multiple sheets for each year (1990-2018).

Walk through excel files (country spreadsheets) and parse selected variables and fuels for each year (sheet in country's spreadsheet).

Somewhere on Eurostat there might be a better source for this data, but I did not find it.

In [1]:
import os
import zipfile
import requests
import pandas as pd
import numpy as np
import pyxlsb

In [2]:
def parse_values_for_country(file, country, variables, fuels):
    """Reads fuel variable in multiple sheets 2002-2018.
    Sums the values across multiple columns if relevant.
    Returns: dict
    """
    country_data = {}
    
    for year in range(2002,2019):
        df = pd.read_excel(
            file,
            engine='pyxlsb',
            sheet_name=str(year),
            skiprows=[0,1,2,3],
            index_col=1,
            na_values=':',
            )
        for variable in variables:
            for fuel, start, end in fuels:             
                try:
                    country_data[(country, year, fuel, variable.lower().replace(' ', '_'))] = df.loc[variable, start:end].sum()
                except TypeError:
                    country_data[(country, year, fuel, variable.lower().replace(' ', '_'))] = pd.to_numeric(df.loc[variable, start:end], errors='coerce').sum()

    return country_data

In [3]:
def walk_through_excel_files(directory, variables, fuels):
    d = {}
    
    for filename in os.listdir(directory):
        if '!' not in filename: # skip readme files
            country = filename.split('-')[0]
            excel_path = os.path.join(directory, filename)
            data = parse_values_for_country(excel_path, country, variables, fuels)
            d.update(data)
    return d

In [4]:
# Selected variables for bioenergy and some other for context
variables = [
'Primary production',
'Imports',
'Exports',
'Gross inland consumption',
]

fuels = [
    ('total', 'Total', 'Total'),
    ('renewables', 'Renewables and biofuels', 'Renewables and biofuels'),
    ('bioenergy', 'Bioenergy', 'Bioenergy',),
    ('solid_biomass', 'Primary solid biofuels', 'Primary solid biofuels'),
    ('biofuels', 'Pure biogasoline', 'Other liquid biofuels'),
    ('biogas', 'Biogases', 'Biogases'),
    ('ren_mun_waste', 'Renewable municipal waste', 'Renewable municipal waste'),
    ]

In [5]:
url = 'https://ec.europa.eu/eurostat/documents/38154/4956218/Energy-Balances-April-2020-edition.zip/69da6e9f-bf8f-cd8e-f4ad-50b52f8ce616'

r = requests.get(url)

with open('eurostat_balances_2020.zip', 'wb') as f:
    f.write(r.content)

In [6]:
with zipfile.ZipFile('eurostat_balances_2020.zip', 'r') as zip_archive:
    zip_archive.extractall(path='balances/')

In [7]:
# This is quite slow, opening many files, one time for each sheet
# There must be a better way

%time data_dict = walk_through_excel_files('balances/', variables, fuels)

CPU times: user 2min 6s, sys: 1.57 s, total: 2min 8s
Wall time: 2min 14s


In [8]:
# https://stackoverflow.com/questions/44012099/creating-a-dataframe-from-a-dict-where-keys-are-tuples
df1 = pd.Series(data_dict).reset_index()
df1.columns = ['country', 'year', 'fuel', 'variable', 'value']

In [9]:
df1.head(3)

,country,year,fuel,variable,value
0,LT,2002,total,primary_production,5227.826
1,LT,2002,renewables,primary_production,772.921
2,LT,2002,bioenergy,primary_production,733.042


In [10]:
df2 = df1.set_index(['country', 'year', 'fuel', 'variable']).unstack(level=3)

In [11]:
df2.head(3)

value                                   \
variable               exports gross_inland_consumption imports   
country year fuel                                                 
AL      2002 bioenergy     0.0                  255.995     0.0   
             biofuels      0.0                    0.000     0.0   
             biogas        0.0                    0.000     0.0   

                                           
variable               primary_production  
country year fuel                          
AL      2002 bioenergy            255.995  
             biofuels               0.000  
             biogas                 0.000

In [12]:
df2.columns = df2.columns.droplevel(0).values

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5117 entries, ('AL', 2002, 'bioenergy') to ('XK', 2018, 'total')
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   exports                   5117 non-null   float64
 1   gross_inland_consumption  5117 non-null   float64
 2   imports                   5117 non-null   float64
 3   primary_production        5117 non-null   float64
dtypes: float64(4)
memory usage: 177.5+ KB


In [14]:
df2.sort_index(ascending=True, inplace=True)

In [15]:
df2['dependency'] = (df2['imports'] - df2['exports']) / df2['gross_inland_consumption']
df2

exports  gross_inland_consumption  imports  \
country year fuel                                                        
AL      2002 bioenergy        0.000                   255.995    0.000   
             biofuels         0.000                     0.000    0.000   
             biogas           0.000                     0.000    0.000   
             ren_mun_waste    0.000                     0.000    0.000   
             renewables       0.000                   559.382    0.000   
...                             ...                       ...      ...   
XK      2018 biogas           0.000                     0.000    0.000   
             ren_mun_waste    0.000                     0.000    0.000   
             renewables       0.000                   403.850    9.141   
             solid_biomass    0.000                   377.312    9.141   
             total           69.604                  2584.929  826.314   

                            primary_production  dependency  
country year fuel                                           
AL      2002 bioenergy                 255.995    0.000000  
             biofuels                    0.000         NaN  
             biogas                      0.000         NaN  
             ren_mun_waste               0.000         NaN  
             renewables                559.382    0.000000  
...                                        ...         ...  
XK      2018 biogas                      0.000         NaN  
             ren_mun_waste               0.000         NaN  
             renewables                394.709    0.022635  
             solid_biomass             368.171    0.024227  
             total                    1822.274    0.292739  

[5117 rows x 5 columns]

In [16]:
df2.to_csv(
    'balances_bioenergy_2002_2018_ktoe.csv',
    decimal=',',
    )

In [17]:
df3 = df2.copy()

In [18]:
tj_ktoe = 41.868

df3 = df3.loc[:, 'exports': 'primary_production'] * tj_ktoe

# Keep the share based on the original data in ktoe
df3['dependency'] = df2['dependency']
df3

exports  gross_inland_consumption  \
country year fuel                                                   
AL      2002 bioenergy         0.000000              10717.998660   
             biofuels          0.000000                  0.000000   
             biogas            0.000000                  0.000000   
             ren_mun_waste     0.000000                  0.000000   
             renewables        0.000000              23420.205576   
...                                 ...                       ...   
XK      2018 biogas            0.000000                  0.000000   
             ren_mun_waste     0.000000                  0.000000   
             renewables        0.000000              16908.391800   
             solid_biomass     0.000000              15797.298816   
             total          2914.180272             108225.807372   

                                 imports  primary_production  dependency  
country year fuel                                                         
AL      2002 bioenergy          0.000000        10717.998660    0.000000  
             biofuels           0.000000            0.000000         NaN  
             biogas             0.000000            0.000000         NaN  
             ren_mun_waste      0.000000            0.000000         NaN  
             renewables         0.000000        23420.205576    0.000000  
...                                  ...                 ...         ...  
XK      2018 biogas             0.000000            0.000000         NaN  
             ren_mun_waste      0.000000            0.000000         NaN  
             renewables       382.715388        16525.676412    0.022635  
             solid_biomass    382.715388        15414.583428    0.024227  
             total          34596.114552        76294.967832    0.292739  

[5117 rows x 5 columns]

In [19]:
df3.to_csv(
    'balances_bioenergy_2002_2018_tj.csv',
    decimal=',',
    )

In [20]:
# Some minimal testing
idx = pd.IndexSlice

In [21]:
df2.loc[idx['CZ', 2018, 'bioenergy'], ['exports']]

exports    549.453
Name: (CZ, 2018, bioenergy), dtype: float64

In [22]:
assert df2.loc[idx['CZ', 2018, 'bioenergy'], ['exports']].item() == 549.453

In [23]:
df2.loc[idx['CZ', 2009, 'bioenergy'], ['primary_production']]

primary_production    2761.8
Name: (CZ, 2009, bioenergy), dtype: float64

In [24]:
assert df2.loc[idx['CZ', 2009, 'bioenergy'], ['primary_production']].item() == 2761.8

In [25]:
result_cz_2009_bioenergy = df2.loc[idx['CZ', 2009, 'bioenergy']]
result_cz_2009_bioenergy

exports                      318.821000
gross_inland_consumption    2568.609000
imports                      123.617000
primary_production          2761.800000
dependency                    -0.075996
Name: (CZ, 2009, bioenergy), dtype: float64

In [26]:
cz_2009_bioenergy = pd.Series(
    {'exports': 318.821,
     'gross_inland_consumption': 2568.609,
     'imports': 123.617,
     'primary_production': 2761.8,
     'dependency': -0.075996,
    })

In [27]:
cz_2009_bioenergy
cz_2009_bioenergy.name = ('CZ', 2009, 'bioenergy')

In [28]:
pd.testing.assert_series_equal(cz_2009_bioenergy, result_cz_2009_bioenergy)